# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8f24b1fc70>
├── 'a' --> tensor([[-1.1307, -0.1037, -0.0327],
│                   [ 1.2484, -1.3821,  0.4463]])
└── 'x' --> <FastTreeValue 0x7f8f24b1fc40>
    └── 'c' --> tensor([[-0.6359, -0.4013,  1.2680, -0.3432],
                        [ 0.7821, -2.2558, -1.0442,  1.3196],
                        [ 0.5749,  0.1544, -0.2772, -0.1607]])

In [4]:
t.a

tensor([[-1.1307, -0.1037, -0.0327],
        [ 1.2484, -1.3821,  0.4463]])

In [5]:
%timeit t.a

76.6 ns ± 0.619 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8f24b1fc70>
├── 'a' --> tensor([[-0.1709, -0.0517, -1.1384],
│                   [ 0.8243,  0.5507, -0.1473]])
└── 'x' --> <FastTreeValue 0x7f8f24b1fc40>
    └── 'c' --> tensor([[-0.6359, -0.4013,  1.2680, -0.3432],
                        [ 0.7821, -2.2558, -1.0442,  1.3196],
                        [ 0.5749,  0.1544, -0.2772, -0.1607]])

In [7]:
%timeit t.a = new_value

80.2 ns ± 1.08 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.1307, -0.1037, -0.0327],
               [ 1.2484, -1.3821,  0.4463]]),
    x: Batch(
           c: tensor([[-0.6359, -0.4013,  1.2680, -0.3432],
                      [ 0.7821, -2.2558, -1.0442,  1.3196],
                      [ 0.5749,  0.1544, -0.2772, -0.1607]]),
       ),
)

In [10]:
b.a

tensor([[-1.1307, -0.1037, -0.0327],
        [ 1.2484, -1.3821,  0.4463]])

In [11]:
%timeit b.a

73.1 ns ± 1.25 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1929,  0.5072,  1.4841],
               [ 0.3934, -0.4201,  0.0521]]),
    x: Batch(
           c: tensor([[-0.6359, -0.4013,  1.2680, -0.3432],
                      [ 0.7821, -2.2558, -1.0442,  1.3196],
                      [ 0.5749,  0.1544, -0.2772, -0.1607]]),
       ),
)

In [13]:
%timeit b.a = new_value

634 ns ± 13.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.07 µs ± 16.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.5 µs ± 147 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

308 µs ± 13 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

281 µs ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8e7a87c9a0>
├── 'a' --> tensor([[[-1.1307, -0.1037, -0.0327],
│                    [ 1.2484, -1.3821,  0.4463]],
│           
│                   [[-1.1307, -0.1037, -0.0327],
│                    [ 1.2484, -1.3821,  0.4463]],
│           
│                   [[-1.1307, -0.1037, -0.0327],
│                    [ 1.2484, -1.3821,  0.4463]],
│           
│                   [[-1.1307, -0.1037, -0.0327],
│                    [ 1.2484, -1.3821,  0.4463]],
│           
│                   [[-1.1307, -0.1037, -0.0327],
│                    [ 1.2484, -1.3821,  0.4463]],
│           
│                   [[-1.1307, -0.1037, -0.0327],
│                    [ 1.2484, -1.3821,  0.4463]],
│           
│                   [[-1.1307, -0.1037, -0.0327],
│                    [ 1.2484, -1.3821,  0.4463]],
│           
│                   [[-1.1307, -0.1037, -0.0327],
│                    [ 1.2484, -1.3821,  0.4463]]])
└── 'x' --> <FastTreeValue 0x7f8e7a87c3a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

51.8 µs ± 4.78 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f8e7a87c2b0>
├── 'a' --> tensor([[-1.1307, -0.1037, -0.0327],
│                   [ 1.2484, -1.3821,  0.4463],
│                   [-1.1307, -0.1037, -0.0327],
│                   [ 1.2484, -1.3821,  0.4463],
│                   [-1.1307, -0.1037, -0.0327],
│                   [ 1.2484, -1.3821,  0.4463],
│                   [-1.1307, -0.1037, -0.0327],
│                   [ 1.2484, -1.3821,  0.4463],
│                   [-1.1307, -0.1037, -0.0327],
│                   [ 1.2484, -1.3821,  0.4463],
│                   [-1.1307, -0.1037, -0.0327],
│                   [ 1.2484, -1.3821,  0.4463],
│                   [-1.1307, -0.1037, -0.0327],
│                   [ 1.2484, -1.3821,  0.4463],
│                   [-1.1307, -0.1037, -0.0327],
│                   [ 1.2484, -1.3821,  0.4463]])
└── 'x' --> <FastTreeValue 0x7f8e7a87c7f0>
    └── 'c' --> tensor([[-0.6359, -0.4013,  1.2680, -0.3432],
                        [ 0.7821, -2.2558, -1.0442,  1.3196],
                 

In [23]:
%timeit t_cat(trees)

43.5 µs ± 2.82 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

83.8 µs ± 2.91 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.1307, -0.1037, -0.0327],
                [ 1.2484, -1.3821,  0.4463]],
       
               [[-1.1307, -0.1037, -0.0327],
                [ 1.2484, -1.3821,  0.4463]],
       
               [[-1.1307, -0.1037, -0.0327],
                [ 1.2484, -1.3821,  0.4463]],
       
               [[-1.1307, -0.1037, -0.0327],
                [ 1.2484, -1.3821,  0.4463]],
       
               [[-1.1307, -0.1037, -0.0327],
                [ 1.2484, -1.3821,  0.4463]],
       
               [[-1.1307, -0.1037, -0.0327],
                [ 1.2484, -1.3821,  0.4463]],
       
               [[-1.1307, -0.1037, -0.0327],
                [ 1.2484, -1.3821,  0.4463]],
       
               [[-1.1307, -0.1037, -0.0327],
                [ 1.2484, -1.3821,  0.4463]]]),
    x: Batch(
           c: tensor([[[-0.6359, -0.4013,  1.2680, -0.3432],
                       [ 0.7821, -2.2558, -1.0442,  1.3196],
                       [ 0.5749,  0.1544, -0.2772, -0.1607]],
         

In [26]:
%timeit Batch.stack(batches)

109 µs ± 2.59 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.1307, -0.1037, -0.0327],
               [ 1.2484, -1.3821,  0.4463],
               [-1.1307, -0.1037, -0.0327],
               [ 1.2484, -1.3821,  0.4463],
               [-1.1307, -0.1037, -0.0327],
               [ 1.2484, -1.3821,  0.4463],
               [-1.1307, -0.1037, -0.0327],
               [ 1.2484, -1.3821,  0.4463],
               [-1.1307, -0.1037, -0.0327],
               [ 1.2484, -1.3821,  0.4463],
               [-1.1307, -0.1037, -0.0327],
               [ 1.2484, -1.3821,  0.4463],
               [-1.1307, -0.1037, -0.0327],
               [ 1.2484, -1.3821,  0.4463],
               [-1.1307, -0.1037, -0.0327],
               [ 1.2484, -1.3821,  0.4463]]),
    x: Batch(
           c: tensor([[-0.6359, -0.4013,  1.2680, -0.3432],
                      [ 0.7821, -2.2558, -1.0442,  1.3196],
                      [ 0.5749,  0.1544, -0.2772, -0.1607],
                      [-0.6359, -0.4013,  1.2680, -0.3432],
                      [ 0.7821, -

In [28]:
%timeit Batch.cat(batches)

198 µs ± 5.87 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

606 µs ± 57.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
